In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [82]:
df = pd.read_csv('train.csv')
df2 = pd.read_csv('train_labels.csv')
test_df = pd.read_csv('test.csv')
geo_df = pd.read_csv('geo_info.csv')
referer_df = pd.read_csv('referer_vectors.csv')
referer_df.head()

,component0;component1;component2;component3;component4;component5;component6;component7;component8;component9;referer
0,16708;-3741;11395;-1597;-3212;6269;5610;-15351...
1,11731;4045;22213;-1184;-8992;9381;-3496;-3120;...
2,10551;2947;12282;-470;16222;4472;-3316;9606;41...
3,12816;20498;-10110;7731;-569;12035;3014;6398;1...
4,3710;11096;11333;14673;8030;1852;10554;11625;4...


In [83]:
referer_columns = referer_df.columns[0].split(';')
new_refs = referer_df['component0;component1;component2;component3;component4;component5;component6;component7;component8;component9;referer'].str.split(';', expand=True)
new_refs.columns = referer_columns
new_refs['referer'] = new_refs['referer'].apply(lambda x: x[8:])
new_refs.head()

,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,referer
0,16708,-3741,11395,-1597,-3212,6269,5610,-15351,13779,14102,a6899a4/15652e67
1,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,9b48ee5/
2,10551,2947,12282,-470,16222,4472,-3316,9606,4197,18948,7a4c700/161af7e3
3,12816,20498,-10110,7731,-569,12035,3014,6398,11439,-271,9653126/159bc361
4,3710,11096,11333,14673,8030,1852,10554,11625,4306,13210,72879b4/125c29e6


In [84]:
dct = df.to_dict()
new_df_columns = list(dct.keys())[0].split(';')[:-1]
info = df.index.tolist()
data = []
for elem in info:
    data.append(elem[0].split(';')[:-1])


In [85]:
new_df2_columns = df2.columns[0].split(';')
new_df2 = df2['user_id;target'].str.split(';', expand=True)
new_df2.columns = new_df2_columns
new_df2

,user_id,target
0,fb858e8e0a2bec074450eaf94b627fd3,0
1,46a5f128fd569c764a92c2eaa788095e,0
2,5a74e9ac53ffb21a20cce117c0ad77ba,0
3,af735816ca19115431ae3d89518c8c91,0
4,364f0ae0a3f29a685c4fb5bae6033b9a,0
...,...,...
499995,11516096d9f97463424523e6154d5255,1
499996,d078ecb7c7e9eaf65189fb3d1bed7871,0
499997,3543d64627ead3a519e3199834e2a148,0
499998,37df5ff1d739f61d442b164db6281e46,0


In [86]:
new_df = pd.DataFrame(data, columns=new_df_columns)
new_df.head()

,request_ts,user_id,referer,geo_id
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,https://9b48ee5/,4799
1,1700986581,46a5f128fd569c764a92c2eaa788095e,https://9b48ee5/,8257
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,https://9634fd0/1409e548,3150
3,1700992803,af735816ca19115431ae3d89518c8c91,https://9b48ee5/,2740
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,https://9b48ee5/,4863


In [ ]:
test_df.head()

,,,request_ts;user_id;referer;geo_id;user_agent
1700993094;c2802dadd33d8ae09bb366bdd41212ea;https://9b48ee5/;8816;{'browser': 'Chrome Mobile','browser_version': '96.0.4664','os': 'Android','os_version': '12'}
1701005579;e5b1988db74527ec092f28b0bbfdaac9;https://9b48ee5/;3663;{'browser': 'Chrome','browser_version': '116.0.5845','os': 'Android','os_version': '10'}
1700969752;6ef1eedbdb72554e53e69782066065c5;https://72879b4/12411b9e;2336;{'browser': 'Chrome','browser_version': '114.0.0','os': 'Android','os_version': '10'}
1700991608;7e057293ecae62985a327b7af51858ea;https://9b48ee5/;9652;{'browser': 'Chrome Mobile','browser_version': '91.0.4472','os': 'Android','os_version': '11'}
1701019815;a27bd7ce8828497823fa8d5d05e7bbf7;https://9b48ee5/;3871;{'browser': 'Chrome Mobile','browser_version': '119.0.0','os': 'Android','os_version': '10'}


In [98]:
dct_test = test_df.to_dict()
new_test_df_columns = list(dct_test.keys())[0].split(';')[:-1]
info_test = test_df.index.tolist()
data_test = []
for elem in info_test:
    data_test.append(elem[0].split(';')[:-1])

test_df_new = pd.DataFrame(data_test, columns=new_test_df_columns)
test_df_new.head()

,request_ts,user_id,referer,geo_id
0,1700993094,c2802dadd33d8ae09bb366bdd41212ea,https://9b48ee5/,8816
1,1701005579,e5b1988db74527ec092f28b0bbfdaac9,https://9b48ee5/,3663
2,1700969752,6ef1eedbdb72554e53e69782066065c5,https://72879b4/12411b9e,2336
3,1700991608,7e057293ecae62985a327b7af51858ea,https://9b48ee5/,9652
4,1701019815,a27bd7ce8828497823fa8d5d05e7bbf7,https://9b48ee5/,3871


In [87]:
geo_df.head()
new_geo_columns = geo_df.columns[0].split(';')
new_geo = geo_df['geo_id;country_id;region_id;timezone_id'].str.split(';', expand=True)
new_geo.columns = new_geo_columns
new_geo.head()

,geo_id,country_id,region_id,timezone_id
0,6447,c31b4e,470e75,f6155e
1,8730,a0a6e9,,d816ca
2,7769,e878d4,,ec4385
3,7330,c31b4e,23f9c2,f6155e
4,600,c31b4e,6dbc37,e56e80


In [ ]:
def prepare_data(df):
    df['referer'] = df['referer'].apply(lambda x: x[8:])
    df[['referer_domain', 'referer_path']] = df['referer'].str.split('/', n=1, expand=True)
    
    df = df.merge(new_refs, on='referer', how='left')
    for i in range(10):
        col = f'component{i}'
        df[col] = pd.to_numeric(df[col]).astype('Int64')

    df = df.merge(new_geo, on='geo_id', how='left')
    
    df['request_ts'] = pd.to_datetime(df['request_ts'], unit='s')
    df['hour'] = df['request_ts'].dt.hour
    df['dow'] = df['request_ts'].dt.dayofweek
    df['is_weekend'] = df['dow'].isin([5, 6]).astype(int)
    
    df['time_of_day'] = pd.cut(
        df['hour'],
        bins=[0, 6, 12, 18, 24],
        labels=['night', 'morning', 'afternoon', 'evening'],
        include_lowest=True
    )
    
    return df

In [89]:
prep = prepare_data(new_df)
prep.head()

C:\Users\Глеб\AppData\Local\Temp\ipykernel_11980\2821380577.py:15: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['request_ts'] = pd.to_datetime(df['request_ts'], unit='s')


,request_ts,user_id,referer,geo_id,referer_domain,referer_path,component0,component1,component2,component3,...,component7,component8,component9,country_id,region_id,timezone_id,hour,dow,is_weekend,time_of_day
0,2023-11-26 15:09:23,fb858e8e0a2bec074450eaf94b627fd3,9b48ee5/,4799,9b48ee5,,11731,4045,22213,-1184,...,-3120,-899,16817,c31b4e,470e75,f6155e,15,6,1,afternoon
1,2023-11-26 08:16:21,46a5f128fd569c764a92c2eaa788095e,9b48ee5/,8257,9b48ee5,,11731,4045,22213,-1184,...,-3120,-899,16817,c31b4e,44520b,e56e80,8,6,1,morning
2,2023-11-26 15:04:31,5a74e9ac53ffb21a20cce117c0ad77ba,9634fd0/1409e548,3150,9634fd0,1409e548,12498,2451,10304,-6380,...,10573,3347,21870,c31b4e,616bb9,af47f1,15,6,1,afternoon
3,2023-11-26 10:00:03,af735816ca19115431ae3d89518c8c91,9b48ee5/,2740,9b48ee5,,11731,4045,22213,-1184,...,-3120,-899,16817,c31b4e,3c9dca,e56e80,10,6,1,morning
4,2023-11-26 18:01:06,364f0ae0a3f29a685c4fb5bae6033b9a,9b48ee5/,4863,9b48ee5,,11731,4045,22213,-1184,...,-3120,-899,16817,c31b4e,776e76,10b7947,18,6,1,afternoon


In [91]:
final_data = prep.merge(new_df2, on='user_id', how='inner')
final_data = final_data.drop('referer', axis=1)
final_data.head()

,request_ts,user_id,geo_id,referer_domain,referer_path,component0,component1,component2,component3,component4,...,component8,component9,country_id,region_id,timezone_id,hour,dow,is_weekend,time_of_day,target
0,2023-11-26 15:09:23,fb858e8e0a2bec074450eaf94b627fd3,4799,9b48ee5,,11731,4045,22213,-1184,-8992,...,-899,16817,c31b4e,470e75,f6155e,15,6,1,afternoon,0
1,2023-11-26 08:16:21,46a5f128fd569c764a92c2eaa788095e,8257,9b48ee5,,11731,4045,22213,-1184,-8992,...,-899,16817,c31b4e,44520b,e56e80,8,6,1,morning,0
2,2023-11-26 15:04:31,5a74e9ac53ffb21a20cce117c0ad77ba,3150,9634fd0,1409e548,12498,2451,10304,-6380,11608,...,3347,21870,c31b4e,616bb9,af47f1,15,6,1,afternoon,0
3,2023-11-26 10:00:03,af735816ca19115431ae3d89518c8c91,2740,9b48ee5,,11731,4045,22213,-1184,-8992,...,-899,16817,c31b4e,3c9dca,e56e80,10,6,1,morning,0
4,2023-11-26 18:01:06,364f0ae0a3f29a685c4fb5bae6033b9a,4863,9b48ee5,,11731,4045,22213,-1184,-8992,...,-899,16817,c31b4e,776e76,10b7947,18,6,1,afternoon,0


In [92]:
X = final_data.drop(['user_id', 'target'], axis=1)
y = final_data['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
cat_features = list(X.select_dtypes(include=['object', 'category']).columns)

In [94]:
cat = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    eval_metric='AUC',
    cat_features=cat_features,
    random_seed=42,
    verbose=100
)
cat.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, use_best_model=True)

0:	test: 0.8347744	best: 0.8347744 (0)	total: 501ms	remaining: 8m 20s
100:	test: 0.8743021	best: 0.8743021 (100)	total: 43.3s	remaining: 6m 25s
200:	test: 0.8778768	best: 0.8778768 (200)	total: 1m 39s	remaining: 6m 34s
300:	test: 0.8794591	best: 0.8794591 (300)	total: 2m 36s	remaining: 6m 2s
400:	test: 0.8804865	best: 0.8804865 (400)	total: 3m 33s	remaining: 5m 19s
500:	test: 0.8811150	best: 0.8811150 (500)	total: 4m 30s	remaining: 4m 29s
600:	test: 0.8816485	best: 0.8816485 (600)	total: 5m 29s	remaining: 3m 38s
700:	test: 0.8820271	best: 0.8820276 (699)	total: 6m 28s	remaining: 2m 45s
800:	test: 0.8823299	best: 0.8823299 (800)	total: 7m 28s	remaining: 1m 51s
900:	test: 0.8825769	best: 0.8825769 (900)	total: 8m 27s	remaining: 55.8s
999:	test: 0.8827450	best: 0.8827450 (999)	total: 9m 27s	remaining: 0us

bestTest = 0.8827450045
bestIteration = 999



In [99]:
prep_test = prepare_data(test_df_new)
prep_test = prep_test.drop('referer', axis=1)
prep_test.head()

C:\Users\Глеб\AppData\Local\Temp\ipykernel_11980\2821380577.py:15: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['request_ts'] = pd.to_datetime(df['request_ts'], unit='s')


,request_ts,user_id,geo_id,referer_domain,referer_path,component0,component1,component2,component3,component4,...,component7,component8,component9,country_id,region_id,timezone_id,hour,dow,is_weekend,time_of_day
0,2023-11-26 10:04:54,c2802dadd33d8ae09bb366bdd41212ea,8816,9b48ee5,,11731,4045,22213,-1184,-8992,...,-3120,-899,16817,c31b4e,36e3f3,f6155e,10,6,1,morning
1,2023-11-26 13:32:59,e5b1988db74527ec092f28b0bbfdaac9,3663,9b48ee5,,11731,4045,22213,-1184,-8992,...,-3120,-899,16817,c31b4e,8ccc01,e56e80,13,6,1,afternoon
2,2023-11-26 03:35:52,6ef1eedbdb72554e53e69782066065c5,2336,72879b4,12411b9e,-7307,11682,9741,13564,13577,...,16461,-3932,3340,c31b4e,1fbfa5,e56e80,3,6,1,night
3,2023-11-26 09:40:08,7e057293ecae62985a327b7af51858ea,9652,9b48ee5,,11731,4045,22213,-1184,-8992,...,-3120,-899,16817,c31b4e,f66ff,f6155e,9,6,1,morning
4,2023-11-26 17:30:15,a27bd7ce8828497823fa8d5d05e7bbf7,3871,9b48ee5,,11731,4045,22213,-1184,-8992,...,-3120,-899,16817,c31b4e,245864,e56e80,17,6,1,afternoon


In [100]:
preds = cat.predict(prep_test.drop('user_id', axis=1))

In [101]:
submit = pd.DataFrame({
    'user_id': prep_test['user_id'],
    'target': preds
})
submit.to_csv('submission.csv', index=False)